In [15]:
import pandas as pd
import json
import numpy as np

In [20]:
all_data = []
for year in range(1991, 2020):
    with open(f'../data/year_data_{year}.json') as f:
        data = json.loads(f.read())
        for comp_cat in data:
            every_row = {
                'year': year,
                'title': comp_cat['title'],
                'date': comp_cat['date'],
                'comp_id': comp_cat['comp_id'],
                'cat_id': comp_cat['cat_id'],
                'type': comp_cat['type']
            }
            row_data = pd.DataFrame(list(map(lambda x: {**x, **every_row}, comp_cat['ranked_athletes'])))
            all_data.append(row_data)

df = pd.concat(all_data, ignore_index=True)
df.to_csv('../data/worldcup_data.csv')
df

,athlete_id,rank,year,title,date,comp_id,cat_id,type
0,266,1,1991,uiaa worldcup - wien 1991,26 April 1991,85,1,lead
1,849,2,1991,uiaa worldcup - wien 1991,26 April 1991,85,1,lead
2,461,3,1991,uiaa worldcup - wien 1991,26 April 1991,85,1,lead
3,973,4,1991,uiaa worldcup - wien 1991,26 April 1991,85,1,lead
4,678,5,1991,uiaa worldcup - wien 1991,26 April 1991,85,1,lead
...,...,...,...,...,...,...,...,...
32622,9941,57,2017,ifsc climbing worldcup (l) - kranj (slo) 2017,11 - 12 November 2017,6843,1,lead
32623,13388,58,2017,ifsc climbing worldcup (l) - kranj (slo) 2017,11 - 12 November 2017,6843,1,lead
32624,13243,59,2017,ifsc climbing worldcup (l) - kranj (slo) 2017,11 - 12 November 2017,6843,1,lead
32625,12702,60,2017,ifsc climbing worldcup (l) - kranj (slo) 2017,11 - 12 November 2017,6843,1,lead
